In [ ]:
import time

date_time = '29.08.2011 11:05:02'
pattern = '%d.%m.%Y %H:%M:%S'
epoch = int(time.mktime(time.strptime(date_time, pattern)))
print epoch

In [25]:
from datetime import datetime,timedelta
import time
import pytz
pattern = '%d.%m.%Y %H:%M:%S'
tz = pytz.timezone('Asia/Hong_Kong')
current_datetime = datetime.now(tz)
cd = current_datetime.strftime('%d.%m.%Y %H:%M:%S')
threehourback_date = datetime.now(tz) - timedelta(hours =3)
tb = threehourback_date.strftime('%d.%m.%Y %H:%M:%S')
current_epoc=int(time.mktime(time.strptime(cd,pattern)))
threehourback_epoc=int(time.mktime(time.strptime(tb,pattern)))

print(threehourback_epoc,current_epoc )

1609873063 1609883863


In [21]:
from datetime import datetime

# current date and time
tz = pytz.timezone('Asia/Hong_Kong')
c_now = datetime.now(tz)

t = c_now.strftime("%d.%m.%Y %H:%M:%S")
print("time:", t)

time: 06.01.2021 03:25:09


In [179]:
import mysql.connector
import csv
zabbixdb = mysql.connector.connect(
  host="10.2.3.9",
  user="dinesh",
  password="Pass1234"
)
#cpu_item = [73031,73129,73227,73325,80463,82749,80534,80605]
cpu_item = {73031:'287332-bvos-app1.bvonesource.com',73129:'287333-bvos-app2.bvonesource.com',73227:'889925-was1.bvonesource.com',73325:'889926-was2.bvonesource.com',80463:'1069313-prod-alf-content1',82749:'1069315-prod-alf-content2',80534:'1069312-prod-alf-index1',80605:'1069314-prod-alf-index2'}
header = ["HOSTNAME","CPU MIN", "CPU AVG", "CPU MAX", "CPU CURRENT"]
f = open('bvos.csv','w')

j = 0
usage = []
usage1 = []
for i in cpu_item:
    c = zabbixdb.cursor(prepared=True)
    c.execute("""select min(value), avg(value), max(value) from (SELECT  value FROM zabbix.history where itemid = %s order by clock desc limit 180) sub""", (i,))
    r = c.fetchall()
    c.execute("""SELECT value FROM zabbix.history where itemid = %s order by clock desc limit 1""", (i,))
    lastvalue = c.fetchall()
    usage = [cpu_item[i],round(r[0][0],2),round(r[0][1],2),round(r[0][2],2),round(lastvalue[0][0],2)]
    usage1.append(usage)
#    print(round(lastvalue[0][0],2),',',round(myresult[0][0],2),',',round(myresult[0][1],2),',',round(myresult[0][2],2) )
#    w = csv.writer(f)
#    w.writerow(usage)

#f.close()
print(usage1)

with open('bvos.csv', 'w',newline='') as f: 
      
    # using csv.writer method from CSV package 
    write = csv.writer(f) 
      
    write.writerow(header) 
    write.writerows(usage1) 
f.close()
    

[['287332-bvos-app1.bvonesource.com', 0.39, 2.05, 14.9, 0.58], ['287333-bvos-app2.bvonesource.com', 0.26, 2.32, 23.5, 0.33], ['889925-was1.bvonesource.com', 0.42, 3.71, 22.01, 2.34], ['889926-was2.bvonesource.com', 0.41, 2.5, 12.88, 2.34], ['1069313-prod-alf-content1', 0.94, 24.63, 83.35, 24.8], ['1069315-prod-alf-content2', 1.2, 8.94, 60.82, 2.72], ['1069312-prod-alf-index1', 1.07, 26.7, 96.99, 25.0], ['1069314-prod-alf-index2', 1.22, 8.21, 52.11, 2.33]]


In [183]:
import mysql.connector
from decimal import *
import csv
zabbixdb = mysql.connector.connect(
  host="10.2.3.9",
  user="dinesh",
  password="Pass1234"
)

heap_item = {77363:'287332-bvos-app1.bvonesource.com',73064:'287333-bvos-app2.bvonesource.com',73162:'889925-was1.bvonesource.com',73260:'889926-was2.bvonesource.com',81037:'1069313-prod-alf-content1',80827:'1069315-prod-alf-content2',81044:'1069312-prod-alf-index1',80774:'1069314-prod-alf-index2'}
header = ["MIN", "AVG", "MAX", "CURRENT"]
f = open('bvos_heap.csv','w')

usage = []
heap_usage = []
for i in heap_item:
    c = zabbixdb.cursor()
    c.execute("""select min(value), avg(value), max(value) from (SELECT value FROM zabbix.history_uint where itemid = %s order by clock desc limit 36) sub""",(i,))
    r = c.fetchall()
    c.execute("""SELECT  value FROM zabbix.history_uint where itemid = %s order by clock desc limit 1""", (i,))
    lastvalue = c.fetchall()
    usage = [round(r[0][0]/(1024*1024*1024),2),round(float(r[0][1]/(1024*1024*1024)),2),round(r[0][2]/(1024*1024*1024),2),round(lastvalue[0][0]/(1024*1024*1024),2)]
    heap_usage.append(usage)

heap_item1 = []
for i in range(len(heap_usage)):
    list2 = []
    if i < 4:
        for j in range(len(heap_usage[i])):
            list2.append(round(heap_usage[i][j]*100/14,2))
    elif i>=4 and i<=5:
         for j in range(len(heap_usage[i])):
            list2.append(round(heap_usage[i][j]*100/8,2))
    else:
        for j in range(len(heap_usage[i])):
            list2.append(round(heap_usage[i][j]*100/12,2))
            
    heap_item1.append(list2)

heap =[]
for i in range(len(heap_item1)):
    list1 = []
    list2 = heap_item1[i]
    list1 = [str(i)+'%' for i in list2]
    heap.append(list1)
    
usage = []
cpu_usage = []
cpu_item = {73031:'287332-bvos-app1.bvonesource.com',73129:'287333-bvos-app2.bvonesource.com',73227:'889925-was1.bvonesource.com',73325:'889926-was2.bvonesource.com',80463:'1069313-prod-alf-content1',82749:'1069315-prod-alf-content2',80534:'1069312-prod-alf-index1',80605:'1069314-prod-alf-index2'}
for i in cpu_item:
    c = zabbixdb.cursor(prepared=True)
    c.execute("""select min(value), avg(value), max(value) from (SELECT  value FROM zabbix.history where itemid = %s order by clock desc limit 180) sub""", (i,))
    r = c.fetchall()
    c.execute("""SELECT value FROM zabbix.history where itemid = %s order by clock desc limit 1""", (i,))
    lastvalue = c.fetchall()
    usage = [round(r[0][0],2),round(r[0][1],2),round(r[0][2],2),round(lastvalue[0][0],2)]
    cpu_usage.append(usage)
    
cpu =[]
for i in range(len(cpu_usage)):
    list1 = []
    list2 = cpu_usage[i]
    list1 = [str(i)+'%' for i in list2]
    cpu.append(list1)


    
usage = []
s_usage = []
solr_item = {80501:'1069312-prod-alf-index1-Workspace-0',81937:'1069312-prod-alf-index1-Workspace-1',80572:'1069314-prod-alf-index2-Workspace-0',81938:'1069314-prod-alf-index2-Workspace-1'}
for i in solr_item:
    c = zabbixdb.cursor()
    c.execute("""select min(value), avg(value), max(value) from( SELECT value FROM zabbix.history_uint where itemid = %s order by clock desc limit 180) sub""",(i,));
    r = c.fetchall()
    c.execute("""SELECT  value FROM zabbix.history_uint where itemid = %s order by clock desc limit 1""", (i,))
    lastvalue = c.fetchall()
    usage = [r[0][0],int(r[0][1]),r[0][2],lastvalue[0][0]]
    s_usage.append(usage)
    
usage = []
db_cpu = []
db_item = {49699:'A1036338-bvos-ora3.bvonesource.com',49999:'A1036343-bvos-ora4.bvonesource.com'}
for i in db_item:
    c = zabbixdb.cursor()
    c.execute("""SELECT min(value),avg(value),max(value) from (select value FROM zabbix.history where itemid = %s order by clock desc limit 180) sub""",(i,));
    r = c.fetchall()
    c.execute("""SELECT  value FROM zabbix.history where itemid = %s order by clock desc limit 1""", (i,))
    lastvalue = c.fetchall()
    usage = [round(r[0][0],2),round(r[0][1],2),round(r[0][2],2),round(lastvalue[0][0],2)]
    db_cpu.append(usage)
    
db_cpu1 =[]
for i in range(len(db_cpu)):
    list1 = []
    list2 = db_cpu[i]
    list1 = [str(i)+'%' for i in list2]
    db_cpu1.append(list1)  
    

usage = []
db_memory = []
db_item = {49710:'A1036338-bvos-ora3.bvonesource.com',50010:'A1036343-bvos-ora4.bvonesource.com'}
a=126.00
for i in db_item:
    c = zabbixdb.cursor()
    c.execute("""SELECT min(value),avg(value),max(value) from (select value FROM zabbix.history_uint where itemid = %s order by clock desc limit 180) sub""",(i,));
    r = c.fetchall()
    c.execute("""SELECT  value FROM zabbix.history_uint where itemid = %s order by clock desc limit 1""", (i,))
    lastvalue = c.fetchall()
    mn = r[0][0]/(1024*1024*1024)
    usage = [round(((a - (r[0][0]/(1024*1024*1024)))/126)*100,2),round(((a - float(r[0][1]/(1024*1024*1024)))/126)*100,2),round(((a - (r[0][2]/(1024*1024*1024)))/126)*100,2),round(((a - (lastvalue[0][0]/(1024*1024*1024)))/126)*100,2)]
    db_memory.append(usage)
    
db_memory1 =[]
for i in range(len(db_memory)):
    list1 = []
    list2 = db_memory[i]
    list1 = [str(i)+'%' for i in list2]
    db_memory1.append(list1)   


usage = []
p_usage = []
db_pool_item = {86417:'BVOS',86415:'ECOM',86414:'NEW_BOOKING',86416:'PTF',86418:'RSM'}
for i in db_pool_item:
    c = zabbixdb.cursor()
    c.execute("""SELECT min(value),avg(value),max(value) from (select value FROM zabbix.history where itemid = %s order by clock desc limit 180) sub""",(i,));
    r = c.fetchall()
    c.execute("""SELECT  value FROM zabbix.history where itemid = %s order by clock desc limit 1""", (i,))
    lastvalue = c.fetchall()
    usage = [int(r[0][0]),int(r[0][1]),int(r[0][2]),int(lastvalue[0][0])]
    p_usage.append(usage)
    
alf_db = ['0','4','12','5']
    
with open('bvos_heap.csv', 'w',newline='') as f: 
      
  
     write = csv.writer(f) 
      
     write.writerow(header) 
     write.writerows(heap)
     write.writerows(db_memory1)
     write.writerows(cpu)
     write.writerows(db_cpu1)
     write.writerows(p_usage)
#     write.writerows(alf_db)
     write.writerows(s_usage)
     print("Data is ready, please check file at your help directory")
     f.close()

Data is ready, please check file at your help directory
